In [1]:
import os

In [2]:
%pwd

'/workspaces/codespaces-blank/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/workspaces/codespaces-blank'

In [18]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/VedantMandre/Chest-Disease-Prediction-Project.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "VedantMandre"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "1220c2f0fb7e0d68a902946464e33951c21968f8"

In [19]:
import tensorflow as tf

In [20]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [21]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [22]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [23]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Chest-CT-Scan-data",
            mlflow_uri="https://dagshub.com/VedantMandre/Chest-Disease-Prediction-Project.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config




In [24]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [30]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")


In [31]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2024-05-15 00:35:40,708: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-05-15 00:35:40,710: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-15 00:35:40,711: INFO: common: created directory at: artifacts]
Found 102 images belonging to 2 classes.


2024-05-15 00:35:41.043744: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


7/7 [==============================] - 37s 5s/step - loss: 9.3727 - accuracy: 0.5686
[2024-05-15 00:36:22,016: INFO: common: json file saved at: scores.json]


2024/05/15 00:36:22 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2024-05-15 00:36:22,805: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: /tmp/tmpsuhiof5w/model/data/model/assets
[2024-05-15 00:36:23,173: INFO: builder_impl: Assets written to: /tmp/tmpsuhiof5w/model/data/model/assets]
[2024-05-15 00:36:26,943: INFO: dir_util: creating /workspaces/codespaces-blank/MLFLOW_TRACKING_URI=https:/dagshub.com/VedantMandre/Chest-Disease-Prediction-Project.mlflow/0/0c327b33d8bc42679edbfa6767c1c46e/artifacts/model/data]
[2024-05-15 00:36:26,944: INFO: dir_util: creating /workspaces/codespaces-blank/MLFLOW_TRACKING_URI=https:/dagshub.com/VedantMandre/Chest-Disease-Prediction-Project.mlflow/0/0c327b33d8bc42679edbfa6767c1c46e/artifacts/model/data/model]
[2024-05-15 00:36:26,947: INFO: dir_

Registered model 'VGG16Model' already exists. Creating a new version of this model...


RestException: RESOURCE_DOES_NOT_EXIST: Response: {'error_code': 'RESOURCE_DOES_NOT_EXIST'}